In [103]:
import torch
import torch.utils.data as Data
from datasets import load_dataset
from transformers import BertTokenizer
from transformers import BertModel
import torch.optim as optim

In [104]:
class Dataset(Data.Dataset):
    """定义数据集"""

    def __init__(self, split):
        self.dataset = load_dataset(path='seamew/ChnSentiCorp', split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        """定义索引方式"""
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']
        return text, label


dataset = Dataset('train')

for text, label in dataset:
    # 调用__getitem__方法
    print(text)
    print(label)
    break

Using custom data configuration default
Reusing dataset chn_senti_corp (C:\Users\duanm\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
1


In [105]:
token = BertTokenizer.from_pretrained('bert-base-chinese')
print(token)

pretrained = BertModel.from_pretrained('bert-base-chinese')
print(pretrained)

# 冻结网络层参数
for param in pretrained.parameters():
    param.requires_grad = False

for weights in pretrained.parameters():
    print(weights.requires_grad)  # 均为False

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    # 批量编码句子
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=512,
                                   return_token_type_ids=True,
                                   return_attention_mask=True,
                                   return_tensors='pt')

    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)
    return input_ids, attention_mask, token_type_ids, labels


loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=32,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

print(len(loader))

for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(input_ids)
    print(input_ids.shape)
    print(attention_mask)
    print(token_type_ids)
    print(labels)
    model_result = pretrained(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids)
    print(model_result.last_hidden_state.shape)
    break

In [108]:
class Model(torch.nn.Module):
    """定义下游训练任务模型"""

    def __init__(self, pretrained_model):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)  # 二分类任务
        self.pretrained = pretrained_model

    def forward(self, input_ids, attention_mask, token_type_ids):
        out = self.pretrained(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids)

        out = self.fc(out.pooler_output)
        out = out.softmax(dim=1)
        return out

In [109]:
# 损失函数
criterion = torch.nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(pretrained)
model = model.to(device)

# 优化器
optimizer = optim.AdamW(model.parameters(), lr=5e-4)

# 训练
model.train()
for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 20 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)
        print(i, loss.item(), accuracy)

    if i == 300:
        break

0 0.7472561597824097 0.34375
20 0.6453174948692322 0.625
40 0.6286224126815796 0.75
60 0.5640804767608643 0.78125
80 0.653367817401886 0.625
100 0.582044780254364 0.6875
120 0.5671188235282898 0.78125
140 0.6375565528869629 0.65625
160 0.4821246266365051 0.875
180 0.49916940927505493 0.8125
200 0.5065711736679077 0.875
220 0.5332591533660889 0.8125
240 0.4951876401901245 0.875
260 0.5902513861656189 0.78125
280 0.5265803933143616 0.75


In [111]:
# 验证
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)
        if i == 5:
            break
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    return correct / total


test()

Using custom data configuration default
Reusing dataset chn_senti_corp (C:\Users\duanm\.cache\huggingface\datasets\seamew___chn_senti_corp\default\0.0.0\1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


0.8